In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [89]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [90]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [91]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [92]:
df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)
df=df.drop(df[df['PrimaryPropertyType']=='University'].index)

In [93]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'SiteEnergyUse_discrete',
       'GHGEmission_discrete', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'SiteEnergyUse(kBtu)_log',
       'GHGEmissions(MetricTonsCO2e)_log'],
      dtype='object')

In [94]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1794, 13)

In [95]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1794, 23)

In [96]:
X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
X_cat.shape

(1794, 36)

In [97]:
#X_cat = np.concatenate((X_cat,df['SiteEnergyUse_discrete'].to_numpy().reshape(-1,1)),axis=1)

In [98]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)']]
X_quant.values.shape

(1794, 4)

In [99]:
X_cat.shape

(1794, 36)

In [100]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [101]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 0)

In [102]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [103]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [104]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: 0.491
R² test sur variable origine: -16.292
R² train sur variable log: 0.547
R² train sur variable orig: -13.34


In [105]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : 0.536
R² train : 0.333


In [106]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [107]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [108]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [109]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [110]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [111]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [112]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4567523343766096e+16, tolerance: 22237159090219.707
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8644744952784320.0, tolerance: 43195090551831.15
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.058806566058813e+16, tolerance: 41827211582145.336
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:52

In [113]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [114]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [115]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [116]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [10, 50, 100, 150, 200],'max_depth':range(5,16, 2), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1,1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart'], 'num_leaves':[30]})

In [117]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),0.53642,8.50046e+06,0.003,{}
Ridge(),0.57518,8.1373e+06,0.001,{'alpha': 100.0}
Lasso(),0.53968,8.47052e+06,0.003005,{'alpha': 100.0}
SVR(kernel='linear'),-0.11686,1.31941e+07,0.056028,"{'C': 4, 'epsilon': 0.1}"
SVR(),-0.11406,1.31775e+07,0.074991,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.15041,1.15076e+07,0.040999,"{'max_depth': 9, 'min_samples_leaf': 4, 'min_s..."
LGBMRegressor(),0.53389,8.5236e+06,0.010998,"{'boosting_type': 'dart', 'learning_rate': 0.1..."


In [118]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),0.22394,276.251,0.001999,{}
Ridge(),0.29455,263.385,0.003,{'alpha': 100.0}
Lasso(),0.25725,270.258,0.000999,{'alpha': 6.135907}
SVR(kernel='linear'),0.23649,274.008,0.063006,"{'C': 4, 'epsilon': 0.6}"
SVR(),0.13598,291.486,0.17104,"{'C': 4, 'epsilon': 0.4, 'kernel': 'sigmoid'}"
RandomForestRegressor(),-0.08119,326.067,0.033003,"{'max_depth': 7, 'min_samples_leaf': 2, 'min_s..."
LGBMRegressor(),0.38876,245.168,0.011999,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [119]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

In [120]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [121]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.228509846605675, tolerance: 0.17596350093570595
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7218243689922019, tolerance: 0.17596350093570595
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2759330973113947, tolerance: 0.17596350093570595
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: C

In [122]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [123]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [124]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [125]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': [10, 50, 100, 150, 200],'max_depth':range(5,16, 2), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1,1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart'], 'num_leaves':[30]})

In [126]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
LinearRegression(),-16.2917,5.19157e+07,0.002001,{}
Ridge(),-13.9068,4.82026e+07,0.002998,{'alpha': 4.037017}
Lasso(),-12.1457,4.52658e+07,0.002001,{'alpha': 0.004977}
SVR(kernel='linear'),-14.1327,4.85667e+07,0.017002,"{'C': 1, 'epsilon': 1.0}"
SVR(),0.55446,8.33342e+06,0.063004,"{'C': 2, 'epsilon': 0.2, 'kernel': 'rbf'}"
RandomForestRegressor(),0.58722,8.02116e+06,0.037003,"{'max_depth': 12, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.58748,8.01867e+06,0.005001,"{'boosting_type': 'gbdt', 'learning_rate': 0.1..."


In [127]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
LinearRegression(),-10.7518,1075,0.002,{}
Ridge(),-7.99759,940.631,0.003008,{'alpha': 6.135907}
Lasso(),-7.98769,940.114,0.003001,{'alpha': 0.003765}
SVR(kernel='linear'),-102.397,3188.68,0.034002,"{'C': 1, 'epsilon': 0.6}"
SVR(),0.33227,256.247,0.059003,"{'C': 1, 'epsilon': 0.4, 'kernel': 'rbf'}"
RandomForestRegressor(),0.36511,249.865,0.042004,"{'max_depth': 14, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.4043,242.03,0.005,"{'boosting_type': 'gbdt', 'learning_rate': 0.1..."
